# Модуль 4

## Разработка REST API на Flask

В качестве задачи выбрана первая задача модуля 3, модель была сохранена через model.save_model("model1",
           format="cbm",
           export_parameters=None,
           pool=None)

### Код обучения модели и загрузки тренировочного датасета

In [ ]:
df_train = pd.read_csv('training_datasets/train_data.csv')

In [ ]:
def ln_x(x):
    return np.log(x)

In [ ]:
def prepare(df_train):
    df_train['feature_1'] = df_train['feature_1'].apply(ln_x)
    Q1 = df_train['feature_3'].quantile(0.25)
    Q3 = df_train['feature_3'].quantile(0.75)
    IQR = Q3 - Q1
    v,n = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR  
    df_train = df_train[(df_train['feature_3'] >= v) & (df_train['feature_3'] <= n)]
    Q1 = df_train['feature_2'].quantile(0.25)
    Q3 = df_train['feature_2'].quantile(0.75)
    IQR = Q3 - Q1
    v,n = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR  
    df_train = df_train[(df_train['feature_2'] >= v) & (df_train['feature_2'] <= n)]
    scaler = MinMaxScaler()
    scaler.fit(df_train[['feature_2', 'feature_3']])
    df_train[['feature_2', 'feature_3']] = scaler.transform(df_train[['feature_2', 'feature_3']])
    return df_train

In [ ]:
df_train = prepare(df_train)

In [ ]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
model.fit(df_train.drop('label', axis = 1), df_train['label'])

In [ ]:
model.save_model("model1",
           format="cbm",
           export_parameters=None,
           pool=None)

Необходимые библиотеки и глобальные переменные

In [ ]:
import os

import requests
from flask import *
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier

info_text = 'Version 1'
help_text ='Доступные методы:<> success() - загрузка данных на сервер<> prepare() - предобработка данных и сохранение в новый файл в сжатом формате<> print_list_datasets() - вывод доступных датасетов<> print_list_models() - вывод доступынх моделей<> predict() - осуществление предсказаний для рабочего датасета с использованием рабочей модели<> setd() - установка рабочего датасета<> setm() - установка рабочей модели<> print_info() - вывести информацию об API'

app = Flask(__name__)
ALLOWED_EXTENSIONS = {'csv'}
work_dataset = 'train_data.pkl'
work_model = 'model1'

## 4.1  Метод для загрузки данных на сервер
Была написана функция success() для загрузки данных. Данные поддерживаются формата csv, если файлы будут неверного формата, файл не будет загружен и выведется сообщение о том, какие форматы доступны.


Для проверки функции была написана доп страница для загрузки данных, html код:

<html>
<head>
    <title>upload the file : GFG</title>
</head>
<body>
    <form action = "/success" method = "post" enctype="multipart/form-data">
        <input type="file" name="file" />
        <input type = "submit" value="Upload">
    </form>
</body>
</html>

main запускается в самом начале и позволяет загружать данные на сервер

In [ ]:
@app.route('/')
def main():
    return render_template("index.html")

Код метода success:

In [ ]:
@app.route('/success', methods=['POST'])
def success():
    if request.method == 'POST':
        f = request.files['file']
        if len(f.filename.split(".")) > 1:
            if f.filename.split(".")[1] in ALLOWED_EXTENSIONS:
                f.save("uploads/" + f.filename)
                return f.filename + ' успешно загружен'
            else:
                return f.filename + ' недопустимого формата, допускаются файлы форматов:' + str(ALLOWED_EXTENSIONS)
        else:
            return f.filename + ' недопустимого формата, допускаются файлы форматов:' + str(ALLOWED_EXTENSIONS)


## 4.2  метод для предобработки данных и сохранения выборки в сжатом формате
Была написана функция prepare() для обработки данных. Полученнный файл сохраняется в формате pickle и выводится его размер в Кб

In [ ]:
def ln_x(x):
    return np.log(x)


def prepare_data(df):
    df['feature_1'] = df['feature_1'].apply(ln_x)
    scaler = MinMaxScaler()
    scaler.fit(df[['feature_2', 'feature_3']])
    df[['feature_2', 'feature_3']] = scaler.transform(df[['feature_2', 'feature_3']])
    return df

@app.route('/prepare',  methods=['GET'])
def prepare():
    name = request.args.get('data')
    if len(name.split(".")) > 1:
        if name.split(".")[1] in ALLOWED_EXTENSIONS:
            df = prepare_data(pd.read_csv('uploads/' + name))
            df.to_pickle('uploads/datasets/' + name.split(".")[0] + '.pkl')
            return str(os.path.getsize('uploads/datasets/' + name.split(".")[0] + '.pkl') / 1000) +' Кбайт занимает полученный датасет'
        else:
            return name + ' недопустимого формата, допускаются файлы форматов:' + str(ALLOWED_EXTENSIONS)
    else:
         return name + ' недопустимого формата, допускаются файлы форматов:' + str(ALLOWED_EXTENSIONS)


## 4.3 Метод для списка доступных наборов данных 
Была написана функция print_list_datasets() для вывода доступных датасетов

In [ ]:
@app.route('/request_data')
def print_list_datasets():
    return os.listdir('uploads/datasets')

## 4.4 Метод для выбора рабочего набора данных
Была написана функция setd() для выбоа рабочего датасета

In [ ]:
@app.route('/setd',  methods=['GET'])
def setd():
    c_dataset = request.args.get('dataset')
    work_dataset = c_dataset
    return work_dataset + ' - текущий рабочий датасет'

## 4.5  Метод для списка доступных моделей 
Была написана функция print_list_models() для вывода всех доступных моделей

In [ ]:
@app.route('/request_model')
def print_list_models():
    return os.listdir('uploads/models')

## 4.6  Метод для выбора модели в качестве рабочей
Была написана функция setm() для выбора рабочей модели

In [ ]:
@app.route('/setm',  methods=['GET'])
def setm():
    c_model = request.args.get('model')
    work_model = c_model
    return work_model + ' - текущая модель'

## 4.7  Метод для выполнения предсказаний при помощи рабочей модели и рабочего набора данных
Была написана функция predict() для получения предсказаний. Работает с рабочей моделью и датасетом

In [ ]:
@app.route('/predict')
def predict():
    df = pd.read_pickle('uploads/datasets/' + work_dataset)
    from_file = CatBoostClassifier()
    from_file.load_model('uploads/models/'+ work_model, format='cbm')
    preds = from_file.predict(df)
    return np.squeeze(preds).tolist()

## 4.9  Метод для вывода информации о имеющихся в API методах 
Была написана функция print_help() для вывода информации о имеющихся методах

In [ ]:
@app.route('/help')
def print_help():
    return help_text.split('<>')

## 4.10  Метод для запроса информации о текущей версии API
Была написана функция print_info() для вывода информации о текущей версии

In [ ]:
@app.route('/info')
def print_info():
    return info_text


# Вывод
В результате выполнени модуля:
 - были написаны методы для основного функционала API
